In [2]:
import pandas as pd

In [3]:
egypt_control = pd.read_csv("../data/egypt_control/experiments.csv")
us_control = pd.read_csv("../data/us_control/experiments.csv")
us_control.describe()
egypt_control.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,6513.000000,6513.000000,6513.000000,6.513000e+03,6513.0,6513.000000,6513.000000,6513.000000
mean,2956.234915,2.262552,14.526793,1.616400e+12,-120.0,0.957470,0.974513,0.791033
std,1856.232904,0.952579,8.698555,7.791336e+07,0.0,0.201811,0.157612,0.406602
min,0.000000,0.000000,0.000000,1.616234e+12,-120.0,0.000000,0.000000,0.000000
25%,1313.000000,2.000000,10.000000,1.616334e+12,-120.0,1.000000,1.000000,1.000000
50%,2941.000000,3.000000,20.000000,1.616439e+12,-120.0,1.000000,1.000000,1.000000
75%,4569.000000,3.000000,21.000000,1.616471e+12,-120.0,1.000000,1.000000,1.000000
max,6197.000000,3.000000,32.000000,1.616503e+12,-120.0,1.000000,1.000000,1.000000


In [4]:
clean_urls_csv = pd.read_csv("../data_collection/urls_to_test.txt", header=None)
clean_urls = list(clean_urls_csv[0])
us_broken_indices = range(6753,6793)

In [5]:
us_control_clean = us_control[(us_control.url.isin(clean_urls) & ~(us_control.experiment_id.isin(us_broken_indices)))]
egypt_control_clean = egypt_control[(egypt_control.url.isin(clean_urls))]

In [6]:
us_control_clean.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,8469.000000,8469.000000,8469.000000,8.469000e+03,8469.0,8469.000000,8469.000000,8469.000000
mean,4565.876727,24.650018,6.757468,1.616367e+12,240.0,0.989727,0.989137,0.986775
std,2864.808267,1.535467,5.402722,8.719841e+07,0.0,0.100839,0.103665,0.114243
min,24.000000,20.000000,0.000000,1.616236e+12,240.0,0.000000,0.000000,0.000000
25%,1582.000000,24.000000,0.000000,1.616303e+12,240.0,1.000000,1.000000,1.000000
50%,4809.000000,25.000000,10.000000,1.616343e+12,240.0,1.000000,1.000000,1.000000
75%,7079.000000,26.000000,11.000000,1.616462e+12,240.0,1.000000,1.000000,1.000000
max,9196.000000,26.000000,28.000000,1.616490e+12,240.0,1.000000,1.000000,1.000000


In [7]:
egypt_control_clean.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,6208.000000,6208.000000,6208.000000,6.208000e+03,6208.0,6208.000000,6208.000000,6208.000000
mean,3039.117429,2.307990,14.589562,1.616404e+12,-120.0,0.971166,0.989691,0.826353
std,1849.818877,0.933345,8.697299,7.710359e+07,0.0,0.167353,0.101018,0.378836
min,0.000000,0.000000,0.000000,1.616234e+12,-120.0,0.000000,0.000000,0.000000
25%,1411.750000,2.000000,10.000000,1.616336e+12,-120.0,1.000000,1.000000,1.000000
50%,3093.500000,3.000000,20.000000,1.616442e+12,-120.0,1.000000,1.000000,1.000000
75%,4645.250000,3.000000,21.000000,1.616473e+12,-120.0,1.000000,1.000000,1.000000
max,6197.000000,3.000000,32.000000,1.616503e+12,-120.0,1.000000,1.000000,1.000000


In [90]:
potential_dns_poisoning = egypt_control_clean[((egypt_control_clean.default_lookup_success == 0) & (egypt_control_clean.cloudflare_lookup_success == 1))]
potential_dns_poisoning_domains = potential_dns_poisoning.domain.unique()
broken_dns_us = us_control_clean[(us_control_clean.domain.isin(potential_dns_poisoning_domains) & (us_control_clean.default_lookup_success == 0))]
dns_poisoned = potential_dns_poisoning[~(potential_dns_poisoning.domain.isin(broken_dns_us.domain))]
dns_poisoned_domains = list(set(potential_dns_poisoning_domains) - set(broken_dns_us.domain))
dns_poisoned_domains
potential_dns_poisoning.domain.unique()


36

In [12]:
potential_blocked = egypt_control_clean[(egypt_control_clean.page_load_success == 0)]
potential_rst = potential_blocked[(potential_blocked.page_load_status.str.startswith('net::ERR_CONNECTION_RESET'))]
potential_rst.experiment_code.unique()

array(['ahmedgamalziada.blogspot.com.eg_0', 'alarab.qa_0', 'alaraby.tv_0',
       'almesryoon.com_0', 'elsharq.web.tv_0', 'fakartany.com_0',
       'hrw.org_0', 'madamasr.com_2', 'qalyubiagate.com_0',
       'sasapost.com_0', 'turk.life_0', 'alscene.com_0', 'al-watan.com_0',
       'amlalommah.net_1', 'ansarportsaed.com_1', 'arabsolaa.com_0',
       'cairoportal.com_1', 'dailysabah.com_0', 'elshaab.org_0',
       'essada.net_0', 'jatoeg.org_0', 'korabia.com_0', 'manalaa.net_0',
       'masrmix.com_0', 'moheet.com_0', 'huffpost.com_0',
       'vvv.almanassa.net_0', 'mada23.appspot.com_0',
       'mada22.appspot.com_0', 'mada21.appspot.com_0',
       'madamirror20.appspot.com_0', 'madamirror18.appspot.com_0',
       'madamirror17.appspot.com_0', 'madamirror16.appspot.com_0',
       'madamirror15.appspot.com_0', 'madamirror14.appspot.com_0',
       'madamirror13.appspot.com_0', 'madamirror12.appspot.com_0',
       'madamirror10.appspot.com_0', 'madamirror9.appspot.com_0',
       'madamirr

In [53]:
empty_resp = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_EMPTY_RESPONSE')]
len(empty_resp.url.unique())

60

In [11]:
connection_refused = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CONNECTION_REFUSED')]
connection_refused

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
25,25,0,egypt_control,dailynewsegypt.com,http://dailynewsegypt.com,0,dailynewsegypt.com_0,1616234294264,-120,"3/20/2021, 11:58:14 AM","[""172.23.176.1""]",success,1,"[""151.139.243.1""]",success,1,"[""151.139.243.1""]","[""151.139.243.1""]",0,net::ERR_CONNECTION_REFUSED at http://dailynew...
1029,714,1,egypt_control,cencos.org,http://cencos.org/,10,cencos.org_10,1616314116152,-120,"3/21/2021, 10:08:36 AM","[""172.23.176.1""]",success,1,"[""159.203.8.241""]",success,1,"[""159.203.8.241""]","[""159.203.8.241""]",0,net::ERR_CONNECTION_REFUSED at http://cencos.org/
3337,3022,3,egypt_control,cencos.org,http://cencos.org/,11,cencos.org_11,1616440379897,-120,"3/22/2021, 9:12:59 PM","[""172.30.48.1""]",success,1,"[""159.203.8.241""]",success,1,"[""159.203.8.241""]","[""159.203.8.241""]",0,net::ERR_CONNECTION_REFUSED at http://cencos.org/
3739,3424,3,egypt_control,madamirror18.appspot.com,https://madamirror18.appspot.com/www.madamasr....,0,madamirror18.appspot.com_0,1616448145838,-120,"3/22/2021, 11:22:25 PM","[""172.30.48.1""]",success,1,"[""172.217.18.244""]",success,1,"[""216.58.198.84""]","[""172.217.18.244"",""216.58.198.84""]",0,net::ERR_CONNECTION_REFUSED at https://madamir...
3923,3608,3,egypt_control,jiameng.com,http://jiameng.com,0,jiameng.com_0,1616451498863,-120,"3/23/2021, 12:18:18 AM","[""172.30.48.1""]",success,1,"[""127.0.0.1""]",success,1,"[""127.0.0.1""]","[""127.0.0.1""]",0,net::ERR_CONNECTION_REFUSED at http://jiameng.com
4140,3825,3,egypt_control,payu.in,http://payu.in,0,payu.in_0,1616455500989,-120,"3/23/2021, 1:25:00 AM","[""172.30.48.1""]",success,1,"[""65.1.34.99"",""13.232.230.23""]",success,1,"[""13.232.230.23"",""65.1.34.99""]","[""65.1.34.99"",""13.232.230.23""]",0,net::ERR_CONNECTION_REFUSED at http://payu.in
4467,4152,3,egypt_control,toolur.com,http://www.toolur.com,21,toolur.com_21,1616462935640,-120,"3/23/2021, 3:28:55 AM","[""172.30.48.1""]",success,1,"[""195.201.209.26""]",success,1,"[""195.201.209.26""]","[""195.201.209.26""]",0,net::ERR_CONNECTION_REFUSED at http://www.tool...
4481,4166,3,egypt_control,toolur.com,http://www.toolur.com/,22,toolur.com_22,1616463226412,-120,"3/23/2021, 3:33:46 AM","[""172.30.48.1""]",success,1,"[""195.201.209.26""]",success,1,"[""195.201.209.26""]","[""195.201.209.26""]",0,net::ERR_CONNECTION_REFUSED at http://www.tool...
6472,6157,3,egypt_control,myworkdayjobs.com,http://myworkdayjobs.com,0,myworkdayjobs.com_0,1616502320389,-120,"3/23/2021, 2:25:20 PM","[""172.30.48.1""]",success,1,"[""127.0.0.1""]",success,1,"[""127.0.0.1""]","[""127.0.0.1""]",0,net::ERR_CONNECTION_REFUSED at http://myworkda...


In [13]:
address_unreachable = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_ADDRESS_UNREACHABLE')]
address_unreachable

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
115,115,0,egypt_control,activewebproxy.cf,http://www.activewebproxy.cf,0,activewebproxy.cf_0,1616235700865,-120,"3/20/2021, 12:21:40 PM","[""172.23.176.1""]",success,1,"[""195.20.51.56""]",success,1,"[""195.20.51.56""]","[""195.20.51.56""]",0,net::ERR_ADDRESS_UNREACHABLE at http://www.act...
159,159,0,egypt_control,bypasstool.gq,http://www.bypasstool.gq,0,bypasstool.gq_0,1616236367310,-120,"3/20/2021, 12:32:47 PM","[""172.23.176.1""]",success,1,"[""195.20.50.134""]",success,1,"[""195.20.50.134""]","[""195.20.50.134""]",0,net::ERR_ADDRESS_UNREACHABLE at http://www.byp...
2721,2406,2,egypt_control,topnewproxy.gq,http://www.topnewproxy.gq,20,topnewproxy.gq_20,1616354052613,-120,"3/21/2021, 9:14:12 PM","[""172.22.160.1""]",success,1,"[""195.20.48.217""]",success,1,"[""195.20.48.217""]","[""195.20.48.217""]",0,net::ERR_ADDRESS_UNREACHABLE at http://www.top...
3540,3225,3,egypt_control,freshfreeproxy.ga,http://www.freshfreeproxy.ga,21,freshfreeproxy.ga_21,1616444243547,-120,"3/22/2021, 10:17:23 PM","[""172.30.48.1""]",success,1,"[""195.20.54.104""]",success,1,"[""195.20.54.104""]","[""195.20.54.104""]",0,net::ERR_ADDRESS_UNREACHABLE at http://www.fre...
4659,4344,3,egypt_control,akproxy.gq,http://www.akproxy.gq,21,akproxy.gq_21,1616466884975,-120,"3/23/2021, 4:34:44 AM","[""172.30.48.1""]",success,1,"[""195.20.52.148""]",success,1,"[""195.20.52.148""]","[""195.20.52.148""]",0,net::ERR_ADDRESS_UNREACHABLE at http://www.akp...
5550,5235,3,egypt_control,bypasstool.gq,http://bypasstool.gq/,22,bypasstool.gq_22,1616484799671,-120,"3/23/2021, 9:33:19 AM","[""172.30.48.1""]",success,1,"[""195.20.50.134""]",success,1,"[""195.20.50.134""]","[""195.20.50.134""]",0,net::ERR_ADDRESS_UNREACHABLE at http://bypasst...
6062,5747,3,egypt_control,takesper.gq,http://www.takesper.gq,22,takesper.gq_22,1616494883340,-120,"3/23/2021, 12:21:23 PM","[""172.30.48.1""]",success,1,"[""195.20.48.69""]",success,1,"[""195.20.48.69""]","[""195.20.48.69""]",0,net::ERR_ADDRESS_UNREACHABLE at http://www.tak...
6432,6117,3,egypt_control,steadyproxy.ga,http://steadyproxy.ga,21,steadyproxy.ga_21,1616501460821,-120,"3/23/2021, 2:11:00 PM","[""172.30.48.1""]",success,1,"[""195.20.52.203""]",success,1,"[""195.20.52.203""]","[""195.20.52.203""]",0,net::ERR_ADDRESS_UNREACHABLE at http://steadyp...


In [17]:
connection_closed = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CONNECTION_CLOSED')]
connection_closed.experiment_code.unique()

array(['madamirror19.appspot.com_0', 'madamirror11.appspot.com_0',
       'madamirrored.appspot.com_0', 'al-sharq.com_0',
       'akhbaralsabah.com_0', 'khatab38.blogspot.com_10',
       'baheyya.blogspot.com_10', 'raseef22.com_10', 'shahidwbas.com_10',
       'eg.arblionz.com_10', 'tv.arblionz.com_10',
       'thepeninsulaqatar.com_10', 'alborsanews.com_20',
       'sasapost.com_20', 'amlalommah.net_20', 'alarab.qa_20',
       'shadowsocks.org_20', 'cdn.ampproject.org_20', 'medium.com_11',
       'baheyya.blogspot.com_11', 'shadowsocks.org_21', 'ida2at.com_22',
       'sasapost.com_22', 'madamirror16.appspot.com_0',
       'arabicpost.net_11', 'elbadil.com_12', 'mada23.appspot.com_0',
       'madamirror14.appspot.com_0', 'amlalommah.net_23',
       'cdn.ampproject.org_21', 'tunnelbear.com_24', 'halacima.net_11',
       'harmees.com_22', 'al-sharq.com_25', 'eipss-eg.org_11',
       'madamirror7.appspot.com_0', 'gamaltaha.wordpress.com_11',
       'shahidwbas.com_11', 'raseef22.com_11',

In [51]:
navigation_timeout_exceeded = potential_blocked[potential_blocked.page_load_status.str.startswith('Navigation timeout')]
navigation_timeout_exceeded_run3 = navigation_timeout_exceeded[navigation_timeout_exceeded.run_number == 3]
us_navigation_timeout = us_control_clean[us_control_clean.page_load_status.str.startswith('Navigation timeout')]
navigation_timeout_exceeded_run3[~(navigation_timeout_exceeded_run3.url.isin(us_navigation_timeout.url))]

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
3118,2803,3,egypt_control,odaily.com,http://odaily.com,0,odaily.com_0,1616436633228,-120,"3/22/2021, 8:10:33 PM","[""172.30.48.1""]",success,1,"[""39.107.149.215""]",success,1,"[""39.107.149.215""]","[""39.107.149.215""]",0,Navigation timeout of 60000 ms exceeded
3124,2809,3,egypt_control,cites.org,https://cites.org/,21,cites.org_21,1616436763288,-120,"3/22/2021, 8:12:43 PM","[""172.30.48.1""]",success,1,"[""104.22.46.86"",""104.22.47.86"",""172.67.25.202""]",success,1,"[""104.22.46.86"",""104.22.47.86"",""172.67.25.202""]","[""104.22.46.86"",""104.22.47.86"",""172.67.25.202""]",0,Navigation timeout of 60000 ms exceeded
3147,2832,3,egypt_control,fhi360.org,https://www.fhi360.org/,21,fhi360.org_21,1616437208284,-120,"3/22/2021, 8:20:08 PM","[""172.30.48.1""]",success,1,"[""23.185.0.3""]",success,1,"[""23.185.0.3""]","[""23.185.0.3""]",0,Navigation timeout of 60000 ms exceeded
3153,2838,3,egypt_control,anonymous-proxy-servers.net,http://anonymous-proxy-servers.net/,0,anonymous-proxy-servers.net_0,1616437332433,-120,"3/22/2021, 8:22:12 PM","[""172.30.48.1""]",success,1,"[""78.129.207.59""]",success,1,"[""78.129.207.59""]","[""78.129.207.59""]",0,Navigation timeout of 60000 ms exceeded
3158,2843,3,egypt_control,childrensdefense.org,http://www.childrensdefense.org/,21,childrensdefense.org_21,1616437446358,-120,"3/22/2021, 8:24:06 PM","[""172.30.48.1""]",success,1,"[""50.115.17.19""]",success,1,"[""50.115.17.19""]","[""50.115.17.19""]",0,Navigation timeout of 60000 ms exceeded
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6448,6133,3,egypt_control,setn.com,http://setn.com,0,setn.com_0,1616501818519,-120,"3/23/2021, 2:16:58 PM","[""172.30.48.1""]",success,1,"[""54.238.119.86""]",success,1,"[""54.238.119.86""]","[""54.238.119.86""]",0,Navigation timeout of 60000 ms exceeded
6466,6151,3,egypt_control,ozon.ru,http://ozon.ru,0,ozon.ru_0,1616502188933,-120,"3/23/2021, 2:23:08 PM","[""172.30.48.1""]",success,1,"[""45.60.40.164"",""45.60.34.164""]",success,1,"[""45.60.34.164"",""45.60.40.164""]","[""45.60.40.164"",""45.60.34.164""]",0,Navigation timeout of 60000 ms exceeded
6476,6161,3,egypt_control,alahlynet.com.eg,http://alahlynet.com.eg,0,alahlynet.com.eg_0,1616502372180,-120,"3/23/2021, 2:26:12 PM","[""172.30.48.1""]",success,1,"[""41.33.192.20""]",success,1,"[""41.33.192.20""]","[""41.33.192.20""]",0,Navigation timeout of 60000 ms exceeded
6489,6174,3,egypt_control,eluniversal.com,http://www.eluniversal.com/,21,eluniversal.com_21,1616502633577,-120,"3/23/2021, 2:30:33 PM","[""172.30.48.1""]",success,1,"[""144.217.180.146""]",success,1,"[""144.217.180.146""]","[""144.217.180.146""]",0,Navigation timeout of 60000 ms exceeded


In [31]:
timeout = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_TIMED_OUT')]
timeout.experiment_code

3139                    opera.com_21
3210                  manalaa.net_12
3218              arab-turkey.com_21
3367                     hamas.ps_13
3391               tunnelbear.com_23
3471                         ok.ru_0
3505              genderation.xyz_11
3538                  tigervpn.com_0
3643                aljazeera.com_13
3687                hidemyass.com_22
3743                   mormon.org_21
3760                     hola.org_23
3819                    turk.life_23
3855                 alalamtv.net_11
3860             vvv.almanassa.net_0
3910                   alhurra.com_0
3940                usacasino.com_21
3956                 ipvanish.com_22
3961                securevpn.com_21
3966                 f-secure.com_21
4034    privateinternetaccess.com_21
4078               online.wsj.com_21
4135             vwv.almanassa.net_0
4494               tunnelbear.com_25
4669                    katib.net_11
4690                shahidlive.co_11
4702     reporter-ohne-grenzen.de_22
4

In [35]:
cert_date_invalid = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CERT_DATE_INVALID')]
cert_date_invalid

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
3600,3285,3,egypt_control,kickstarter.com,http://kickstarter.com,0,kickstarter.com_0,1616445334801,-120,"3/22/2021, 10:35:34 PM","[""172.30.48.1""]",success,1,"[""151.101.2.217"",""151.101.66.217"",""151.101.130...",success,1,"[""151.101.66.217"",""151.101.194.217"",""151.101.1...","[""151.101.2.217"",""151.101.66.217"",""151.101.130...",0,net::ERR_CERT_DATE_INVALID at http://kickstart...


In [39]:
too_many_redirects = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_TOO_MANY_REDIRECTS')]
too_many_redirects

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
4333,4018,3,egypt_control,state.gov,https://www.state.gov/j/drl/,23,state.gov_23,1616459879718,-120,"3/23/2021, 2:37:59 AM","[""172.30.48.1""]",success,1,"[""34.233.79.178""]",success,1,"[""34.233.79.178""]","[""34.233.79.178""]",0,net::ERR_TOO_MANY_REDIRECTS at https://www.sta...


In [43]:
aborted = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_ABORTED')]
us_control_clean[us_control_clean.domain == 'denetsuk.com']

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
7418,7605,26,us_control,denetsuk.com,http://denetsuk.com,0,denetsuk.com_0,1616469105823,240,"3/22/2021, 11:11:45 PM","[""172.27.64.1""]",success,1,"[""139.45.197.101"",""139.45.197.90""]",success,1,"[""139.45.197.101"",""139.45.197.90""]","[""139.45.197.101"",""139.45.197.90""]",0,net::ERR_ABORTED at http://denetsuk.com


In [47]:
cert_name_invalid = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CERT_COMMON_NAME_INVALID')]
us_control_clean[us_control_clean.domain == 'khales.com.eg']

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
8462,8649,26,us_control,khales.com.eg,http://khales.com.eg,0,khales.com.eg_0,1616483200078,240,"3/23/2021, 3:06:40 AM","[""172.27.64.1""]",success,1,"[""196.46.22.61""]",success,1,"[""196.46.22.61""]","[""196.46.22.61""]",0,net::ERR_CERT_COMMON_NAME_INVALID at http://kh...


In [81]:
nonblocked_domains = egypt_control_clean[((egypt_control_clean.page_load_success == 1) & ~(egypt_control_clean.url.isin(potential_blocked.url)) & (egypt_control_clean.default_lookup_success == 1) & ~(egypt_control_clean.domain.isin(dns_poisoned_domains)))]
nonblocked_domains.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,4853.000000,4853.000000,4853.000000,4.853000e+03,4853.0,4853.0,4853.0,4853.0
mean,3110.267257,2.369668,15.120956,1.616407e+12,-120.0,1.0,1.0,1.0
std,1824.429394,0.865765,8.527295,7.464248e+07,0.0,0.0,0.0,0.0
min,2.000000,0.000000,0.000000,1.616234e+12,-120.0,1.0,1.0,1.0
25%,1516.000000,2.000000,10.000000,1.616338e+12,-120.0,1.0,1.0,1.0
50%,3171.000000,3.000000,20.000000,1.616443e+12,-120.0,1.0,1.0,1.0
75%,4710.000000,3.000000,21.000000,1.616474e+12,-120.0,1.0,1.0,1.0
max,6197.000000,3.000000,32.000000,1.616503e+12,-120.0,1.0,1.0,1.0


In [109]:
# Generate list of domains
normal_sample = list(nonblocked_domains.sample(n=30).url)
dns_poisoned_sample = list(dns_poisoned.sample(n=30).url)
rst_packet_sample = list(potential_rst.sample(n=30).url)
empty_rsp_sample = list(empty_resp.sample(n=30).url)
conn_closed_sample = list(connection_closed.sample(n=30).url)
timed_out_sample = list(timeout.sample(n=30).url)
conn_refused_urls = list(connection_refused.url)
address_unreachable_urls = list(address_unreachable.url)
cert_date_invalid_url = list(cert_date_invalid.url)
too_many_redirects_url = list(too_many_redirects.url)
sample_urls = list(set(normal_sample + dns_poisoned_sample + rst_packet_sample + empty_rsp_sample + conn_closed_sample + timed_out_sample + conn_refused_urls + address_unreachable_urls + cert_date_invalid_url + too_many_redirects_url))

In [117]:
len(sample_urls)
with open("./../data_collection/sample_urls.txt", "w") as f:
    f.write('\n'.join(sample_urls))